In [1]:
%reload_ext autoreload
%autoreload 2

import vtk
from vtk.util import numpy_support

import numpy as np
import bloscpack as bp

import sys
import os
sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *
from annotation_utilities import *
from registration_utilities import *

from skimage.measure import mesh_surface_area, marching_cubes, correct_mesh_orientation

from itertools import izip
import pandas as pd

import time

import matplotlib.pyplot as plt
%matplotlib inline

from vis3d_utilities import *

Setting environment for Local Macbook Pro


In [2]:
with open('colors.txt', 'r') as f:
    colors = {l[0]: np.r_[float(l[1]), float(l[2]), float(l[3])] for l in map(lambda x: x.split(), f.readlines())}

In [3]:
atlas_volume = bp.unpack_ndarray_file(volume_dir + '/atlasVolume_icp.bp').astype(np.int8)

available_labels_sided = [labels_sided[i-1] for i in np.unique(atlas_volume) if i > 0]
available_labels_unsided = set([labelMap_sidedToUnsided[name] for name in available_labels_sided ])

IOError: [Errno 2] No such file or directory: '/home/yuncong/CSHL_volumes2//atlasVolume_icp.bp'

In [5]:
stack = 'MD594'

In [6]:
# Load atlasProjected volume

atlasProjected_polydata_list = {}

for name_s in available_labels_sided:

    fn = mesh_rootdir + "/%(stack)s/%(stack)s_atlasProjectedVolume_%(name)s_smoothed.stl" % {'stack': stack, 'name': name_s}

    reader = vtk.vtkSTLReader()
    reader.SetFileName(fn)
    reader.Update()

    polydata = reader.GetOutput()    
    atlasProjected_polydata_list[name_s] = polydata

In [7]:
# Load localAdjusted volume

localAdjusted_polydata_list = {}

for name_s in available_labels_sided:

    fn = mesh_rootdir + "/%(stack)s/%(stack)s_localAdjustedVolume_%(name)s_smoothed.stl" % {'stack': stack, 'name': name_s}

    reader = vtk.vtkSTLReader()
    reader.SetFileName(fn)
    reader.Update()

    polydata = reader.GetOutput()    
    localAdjusted_polydata_list[name_s] = polydata
    
    
centroid_localAdjusted_polydata_list = {}

for name_s, polydata in localAdjusted_polydata_list.iteritems():
    vertices, faces = polydata_to_mesh(polydata)
    centroid_localAdjusted_polydata_list[name_s] = vertices.mean(axis=0)

In [8]:
hessian_allStacks_allLandmarks = pickle.load(open('/home/yuncong/CSHL_atlasAlignParams_atlas/hessian_individualAlign_allStacks_allLandmarks.pkl', 'r'))
std_allStacks_allLandmarks = pickle.load(open('/home/yuncong/CSHL_atlasAlignParams_atlas/std_individualAlign_allStacks_allLandmarks.pkl', 'r'))

In [ ]:
# hessian_allLandmarks = {name_s: stk_v[stack] for name_s, stk_v in hessian_allStacks_allLandmarks.iteritems()}
# std_allLandmarks = {name_s: stk_v[stack] for name_s, stk_v in std_allStacks_allLandmarks.iteritems()}

In [8]:
# matplotlib sphere plot

phi, theta = np.mgrid[0:np.pi:51j, 0:2 * np.pi:51j]
sphere_xs = np.sin(phi) * np.cos(theta)
sphere_ys = np.sin(phi) * np.sin(theta)
sphere_zs = np.cos(phi)
sphere_vecs = np.c_[sphere_xs.flat, sphere_ys.flat, sphere_zs.flat]
# sphere_vecs = np.dstack([x, y, z])

from matplotlib import colors

def show_sphere_plot(fcolors, title=None):
    
#     from matplotlib import colors
    from mpl_toolkits.mplot3d import Axes3D
    
    fig = plt.figure(figsize=plt.figaspect(1.))
    ax = fig.add_subplot(111, projection='3d')
    
    norm = colors.Normalize(vmin = 0, vmax = 2.5, clip = False)
    m = plt.cm.ScalarMappable(norm=norm, cmap=plt.cm.jet)
    m.set_array(fcolors)
    
    surf = ax.plot_surface(sphere_xs, sphere_ys, sphere_zs,  rstride=1, cstride=1, 
                           facecolors=m.to_rgba(fcolors))
    # Turn off the axis planes
    ax.set_axis_off()
    if title is not None:
        plt.title(title);
    
    plt.colorbar(m);
    plt.show()

%matplotlib qt4
show_sphere_plot(np.array([1./2. * 10**2 * np.dot(np.dot(sv, Htr[name_s][stack]), sv) / std_allStacks_allLandmarks[name_s][stack]
          for sv in sphere_vecs]).reshape((sphere_xs.shape)))

NameError: name 'Htr' is not defined

In [9]:
# VTK sphere plot

In [32]:
length_factor = 1e6

R = defaultdict(dict)
V = defaultdict(dict)
S = defaultdict(dict)
Htr = defaultdict(dict)

for li, (name_s, hs) in enumerate(hessian_allStacks_allLandmarks.iteritems()):

    for st, h in hs.iteritems():
        
        std = std_allStacks_allLandmarks[name_s][st]
        
        q = np.r_[3,7,11]
        h_tr = h[q[:,None], q]
        
        Htr[name_s][st] = h_tr
        
        s, v = np.linalg.eigh(h_tr)
                
        S[name_s][st] = 1./2. * 10**2 * s[2] / std
        V[name_s][st] = v

        R[name_s][st] = v*2e-9/s[None, :] * length_factor
        R[name_s][st] = R[name_s][st] * np.sign(R[name_s][st][0])
        
R.default_factory = None
V.default_factory = None
S.default_factory = None
Htr.default_factory = None

In [14]:
mesh_actors = [actor_mesh(polydata, (1.,1.,1.), opacity=.3, wireframe=True) 
               for name_s, polydata in localAdjusted_polydata_list.iteritems() if name_s == 'VLL_R']

sphere_actors = [actor_mesh(polydata_heat_sphere(lambda v: -1./2. * 10**2 * np.dot(np.dot(v, Htr[name_s][stack]), v) / std_allStacks_allLandmarks[name_s][stack], 
                                            loc=centroid_localAdjusted_polydata_list[name_s], radius=20,
                                           vmin=0.5, vmax=2.5))
           for name_s in available_labels_sided if name_s == 'VLL_R']

launch_vtk(mesh_actors + sphere_actors)

In [12]:
mesh_actors = [actor_mesh(polydata, (1.,1.,1.), opacity=.3, wireframe=True) 
               for name_s, polydata in localAdjusted_polydata_list.iteritems()]

sphere_actors = [actor_mesh(polydata_heat_sphere(lambda v: -1./2. * 10**2 * np.dot(np.dot(v, Htr[name_s][stack]), v) / std_allStacks_allLandmarks[name_s][stack], 
                                            loc=centroid_localAdjusted_polydata_list[name_s], radius=20,
                                           vmin=0.5, vmax=2.5))
           for name_s in available_labels_sided]

launch_vtk(mesh_actors + sphere_actors)

In [35]:
anchor_points = [ centroid_localAdjusted_polydata_list[name_s] for name_s in ['VLL_R']]
anchor_vectors2 = [ R[name_s][stack][:,2] for name_s in ['VLL_R']]
anchor_vectors1 = [ R[name_s][stack][:,1] for name_s in ['VLL_R']]
anchor_vectors0 = [ R[name_s][stack][:,0] for name_s in ['VLL_R']]

In [33]:
anchor_points = [ centroid_localAdjusted_polydata_list[name_s] for name_s in available_labels_sided]
anchor_vectors2 = [ R[name_s][stack][:,2] for name_s in available_labels_sided]
anchor_vectors1 = [ R[name_s][stack][:,1] for name_s in available_labels_sided]
anchor_vectors0 = [ R[name_s][stack][:,0] for name_s in available_labels_sided]

In [26]:
# Show uncertainty ellipse

# mesh_actors = [actor_mesh(polydata, colors[labelMap_sidedToUnsided[name_s]], opacity=.3, wireframe=True) 
#                for name_s, polydata in localAdjusted_polydata_list.iteritems()]

mesh_actors = [actor_mesh(polydata, (1.,1.,1.), opacity=.3, wireframe=True) 
               for name_s, polydata in localAdjusted_polydata_list.iteritems()]

ellipse_actors = [actor_ellipse(anchor_point, anchor_vector0, anchor_vector1, anchor_vector2, wireframe=True)
                  for anchor_point, anchor_vector0, anchor_vector1, anchor_vector2 \
                  in zip(anchor_points, anchor_vectors0, anchor_vectors1, anchor_vectors2)]

launch_vtk(mesh_actors + ellipse_actors, init_angle='30')

In [ ]:
# Show uncertainty arrows

mesh_actors = [actor_mesh(polydata, (1.,1.,1.), opacity=.3, wireframe=True) 
               for name_s, polydata in localAdjusted_polydata_list.iteritems() if name_s == 'VLL_R']

arrow_actors = list(chain(*[actor_arrows(anchor_point, anchor_vector0, anchor_vector1, anchor_vector2)
                  for anchor_point, anchor_vector0, anchor_vector1, anchor_vector2 \
                  in zip(anchor_points, anchor_vectors0, anchor_vectors1, anchor_vectors2)]))

launch_vtk(mesh_actors + arrow_actors, init_angle='30')

In [25]:
# Show three axes

ren1 = vtk.vtkRenderer()

renWin1 = vtk.vtkRenderWindow()
renWin1.AddRenderer(ren1)

iren1 = vtk.vtkRenderWindowInteractor()
iren1.SetRenderWindow(renWin1)

camera = vtk.vtkCamera()

# 30 deg
camera.SetViewUp(0, -1, 0)
camera.SetPosition(-10, -5, -5)
camera.SetFocalPoint(1, 1, 1)

# # 45 degree
# camera.SetViewUp(0, -1, 0)
# camera.SetPosition(-20, -30, -10)
# camera.SetFocalPoint(1, 1, 1)

# # saggital
# camera.SetViewUp(0, -1, 0)
# camera.SetPosition(0, 0, -2)
# camera.SetFocalPoint(0, 0, 1)

# # coronal
# camera.SetViewUp(0, -1, 0)
# camera.SetPosition(-2, 0, 0)
# camera.SetFocalPoint(-1, 0, 0)

# # horizontal
# camera.SetViewUp(0, 0, -1)
# camera.SetPosition(0, 1, 0)
# camera.SetFocalPoint(0, -1, 0)
    
# for i, (name_s, polydata) in enumerate(localAdjusted_polydata_list.iteritems()):
    
#     m = vtk.vtkPolyDataMapper()
#     m.SetInputData(polydata)

#     a = vtk.vtkActor()
#     a.SetMapper(m)
#     a.GetProperty().SetRepresentationToWireframe()
# #     a.GetProperty().SetColor(colors[labelMap_sidedToUnsided[name_s]])
#     a.GetProperty().SetColor((1.,1.,1.))
#     a.GetProperty().SetOpacity(.1)
    
#     ren1.AddActor(a)
    
    
arrowSource = vtk.vtkArrowSource()

for anchor_vectors, c in zip([anchor_vectors2, anchor_vectors1, anchor_vectors0],
                            [(1.,0.,0.),(0.,1.,0.),(0.,0.,1.)]):

    for p_ind in range(len(anchor_points)):

        anchor_point = anchor_points[p_ind]
        anchor_vector = anchor_vectors[p_ind]

        length = np.linalg.norm(anchor_vector)
        normalizedX = anchor_vector/length

        arbitrary = np.random.uniform(-10, 10, 3)
        normalizedZ = np.cross(normalizedX, arbitrary)
        normalizedZ = normalizedZ/np.linalg.norm(normalizedZ)
        normalizedY = np.cross(normalizedZ, normalizedX)
        normalizedY = normalizedY/np.linalg.norm(normalizedY)

        matrix = vtk.vtkMatrix4x4()

        # Create the direction cosine matrix
        matrix.Identity()
        for i in range(3):
            matrix.SetElement(i, 0, normalizedX[i])
            matrix.SetElement(i, 1, normalizedY[i])
            matrix.SetElement(i, 2, normalizedZ[i])

        # Apply the transforms
        transform = vtk.vtkTransform()
        transform.Translate(anchor_point)
        transform.Concatenate(matrix)
        transform.Scale(length, length, length)

        # Transform the polydata
        transformPD = vtk.vtkTransformPolyDataFilter()
        transformPD.SetTransform(transform)
        transformPD.SetInputConnection(arrowSource.GetOutputPort())

        #Create a mapper and actor for the arrow
        m = vtk.vtkPolyDataMapper()
        m.SetInputConnection(transformPD.GetOutputPort())

        a = vtk.vtkActor()
        a.SetMapper(m)

        a.GetProperty().SetColor(c)

        ren1.AddActor(a)
    
    
for name in ['VLL']:

    score_volume = bp.unpack_ndarray_file(volume_dir + '/%(stack)s/%(stack)s_scoreVolume_%(name)s.bp' % \
                                          {'stack':stack, 'name':name})
    score_imagedata = volume_to_imagedata(score_volume)

    volumeMapper = vtk.vtkSmartVolumeMapper()
    #     volumeMapper.SetBlendModeToComposite()
    volumeMapper.SetInputData(score_imagedata)

    volumeProperty = vtk.vtkVolumeProperty()
    #     volumeProperty.ShadeOff()
    # volumeProperty.SetInterpolationType(vtk.VTK_LINEAR_INTERPOLATION)

    compositeOpacity = vtk.vtkPiecewiseFunction()
    compositeOpacity.AddPoint(0.0, 0.0)
    compositeOpacity.AddPoint(0.95, 0.01)
    compositeOpacity.AddPoint(1.0, 1.0)
    volumeProperty.SetScalarOpacity(compositeOpacity)

    color = vtk.vtkColorTransferFunction()
#     c = colors[name]
    c = (1., 1., 1.)
    color.AddRGBPoint(0.0, c[0], c[1], c[2])
    color.AddRGBPoint(255.0, c[0], c[1], c[2])
    volumeProperty.SetColor(color)

    a = vtk.vtkVolume()
    a.SetMapper(volumeMapper)
    a.SetProperty(volumeProperty)
    
    ren1.AddActor(a)
    
    
ren1.SetActiveCamera(camera)

widget1 = add_axes(iren1)
renWin1.Render()

ren1.ResetCamera()
renWin1.SetWindowName('original')

iren1.Start()

In [ ]:
# Show three axes, glyph3D approach

vector_map = np.zeros((10,10,10,3))
for p_ind in range(len(anchor_points)):
    x,y,z = anchor_points[p_ind]
    vector_map[y,x,z] = anchor_vectors[p_ind]
    
arrowSource = vtk.vtkArrowSource()
arrowSource.Update()

glyphFilter = vtk.vtkGlyph3D()
glyphFilter.SetSourceConnection(arrowSource.GetOutputPort())
glyphFilter.OrientOn()
glyphFilter.SetVectorModeToUseVector()
glyphFilter.SetScaleModeToScaleByVector()
# glyphFilter.SetScaleModeToDataScalingOff()
glyphFilter.SetColorModeToColorByScalar()

imagedata = vectormap_to_imagedata(vector_map, colors=[255,255,255])
glyphFilter.SetInputData(imagedata)
glyphFilter.Update()

m = vtk.vtkPolyDataMapper()
m.SetInputConnection(glyphFilter.GetOutputPort())
m.ScalarVisibilityOn()

ren1 = vtk.vtkRenderer()

renWin1 = vtk.vtkRenderWindow()
renWin1.AddRenderer(ren1)

iren1 = vtk.vtkRenderWindowInteractor()
iren1.SetRenderWindow(renWin1)

camera = vtk.vtkCamera()

# 45 degree
camera.SetViewUp(0, -1, 0)
camera.SetPosition(-20, -30, -10)
camera.SetFocalPoint(1, 1, 1)

# # saggital
# camera.SetViewUp(0, -1, 0)
# camera.SetPosition(0, 0, -2)
# camera.SetFocalPoint(0, 0, 1)

# # coronal
# camera.SetViewUp(0, -1, 0)
# camera.SetPosition(-2, 0, 0)
# camera.SetFocalPoint(-1, 0, 0)

# # horizontal
# camera.SetViewUp(0, 0, -1)
# camera.SetPosition(0, 1, 0)
# camera.SetFocalPoint(0, -1, 0)


# m = vtk.vtkPolyDataMapper()
# m.SetInputData(polydata)

a = vtk.vtkActor()
a.SetMapper(m)
# a.GetProperty().SetRepresentationToWireframe()
#     a.GetProperty().SetColor(colors[labelMap_sidedToUnsided[name_s]])
# a.GetProperty().SetColor((1.,1.,1.))

ren1.AddActor(a)

ren1.SetActiveCamera(camera)

widget1 = add_axes(iren1)
renWin1.Render()

ren1.ResetCamera()
renWin1.SetWindowName('original')

iren1.Start()